### Notice, running this code need a codex api request from openai

In [1]:
import os
import openai
import string
from utils import *
from www.dataset.ann import *
from tqdm import tqdm
import time
%load_ext autoreload
%autoreload 2
#openai.api_key = 


In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preparation

#### 1) get Tripdataset

In [34]:
from attribute import * 

In [1]:
data_file = ('../../Source_task/trip.json')
with open(data_file, 'r') as f:
    cloze_dataset_2s, order_dataset_2s = json.load(f)  

for p in cloze_dataset_2s:
    label_dist = Counter([ex['label'] for ex in cloze_dataset_2s[p]])
    print('Cloze label distribution (%s):' % p)
    print(label_dist.most_common())

tiered_dataset = cloze_dataset_2s


maxStoryLength=168       
tiered_dataset = get_tiered_data(tiered_dataset)
tiered_dataset = add_bert_features_tiered_modify(tiered_dataset, robertatokenizer,maxStoryLength, add_segment_ids=True)

## Presetting function

In [31]:
def KATE(Sentence,Entity,human_flag = False):
    for name in Name_list:
        Sentence = Sentence.replace(name,"Someone")
        
    Target_Sentence = Sentence+" Focus: "+Entity+"."
    target_output = SentenceTransformer.encode(Target_Sentence,output_value = None)
    target_sentenceemb = target_output['sentence_embedding'].cpu().numpy()
    target_wordemb = get_wordemb(target_output)
    if human_flag:
        for key in attribute_sen.keys():
            for sen_output in attribute_sen[key]:
                sentence =sen_output['sentence']
                original_sen = sentence.split(".")[0]
                entity_name = sentence.split(":")[-1].split('.')[0]
                sen_emb = sen_output['sentence_embedding']
                word_emb = sen_output['word_embeddings']
                sen_cos = computeCos(target_sentenceemb,sen_emb)
                word_cos = computeCos(target_wordemb,word_emb)
                sen_output['Cos_similarity'] = 0.6*sen_cos+0.4*word_cos
    else:
        for key in attribute_sen.keys():
            for sen_output in attribute_sen[key]:
                sentence =sen_output['sentence']
                original_sen = sentence.split(".")[0]
                entity_name = sentence.split(":")[-1].split('.')[0]
                if entity_name in original_sen:
                    sen_emb = sen_output['sentence_embedding']
                    word_emb = sen_output['word_embeddings']
                    sen_cos = computeCos(target_sentenceemb,sen_emb)
                    word_cos = computeCos(target_wordemb,word_emb)
                    sen_output['Cos_similarity'] = 0.6*sen_cos+0.4*word_cos
                    
                    
    distance_list  = []      
    min_length = min([len(attribute_sen[key]) for key in attribute_sen.keys()])
    min_length = min(4,min_length)
    for key in attribute_sen.keys():
        temp_list = attribute_sen[key]
        temp_list = sorted(temp_list, key = lambda i : (-i['Cos_similarity']))
        for i in range(min_length):
            temp_pro = temp_list[i]
            temp_pro['attribute_label'] = int(key)
            distance_list.append(temp_pro)

    
    return distance_list


def singleAttributePrompting(sentence,entity,human_flag,output_log):
    local_count=30
#     print(sentence,entity)
    while True:
        try:
            
            
            prompting_list  = KATE(sentence,entity,human_flag)
            score_list = [_['Cos_similarity'] for _ in prompting_list]
            key_list = set([_['attribute_label'] for _ in prompting_list])
            score_list = np.array(score_list)
            total_length = len(key_list)
            max_value = np.max(score_list) 
            max_idx = np.where(score_list == max_value)[0][0]
            label = None
            if max_value > 0.95:
                label = int(max_idx/total_length)   
                return label
            else:
                sentence_prompting = ""
                for i in range(len(prompting_list)):
                    prompinting_temp = prompting_list[i]
                    original_sentence = prompinting_temp['sentence']
                    original_sen = original_sentence.split(".")[0]+"."
                    entity_name = original_sentence.split(":")[-1].split('.')[0]
                    attribute_label = prompinting_temp['attribute_label']
                    sentence_prompting += getSingleprompt(attribute,(original_sen,entity_name),attribute_label)+'\n\n'


                input_prompting_entity = getSingleprompt(attribute,(sentence,entity),'_',input_text=True)
                input_prompting_entity = sentence_prompting + input_prompting_entity
    #             print(input_prompting_entity+'\n\n\n')
                response = openai.Completion.create(
                  model="code-davinci-002",
                  prompt= input_prompting_entity,
                  temperature=0,
                  max_tokens=1000,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0,
                  stop = '#END'
                )

                response_clean = response['choices'][0]['text']
                activate_label = int(response_clean.split('=')[1].split('\n')[0].strip())
                if total_length ==9:
                    if activate_label>8:
                        activate_label =8
                if total_length == 3:
                    if activate_label>2:
                        activate_label =2                    
                return int(activate_label)

        except Exception as e:
            time.sleep(local_count)
            print("error! wait {local_count}s".format(local_count=local_count))
            with open(output_log,"a") as file:
                file.write("error! wait {local_count}s\n".format(local_count=local_count))
#                 file.write(str(e))
            continue
    

### CSKG

loading data

In [7]:
Name_list = np.load('Name_list.npy',allow_pickle = True)
cskg_data = np.load('../../Source_task/cskg_data.npy',allow_pickle = True)

main pipepline of prompting

In [26]:
def PromptingPipe_CSKG(dev_dataset,attribute,activate_list,attribute_label_list,log_name,human_flag = False):
    attribute_id = att_to_idx[attribute]
    Pred_summary = []
    count = 0
    output_log = os.path.join(log_name,"{}_log.txt".format(attribute))
#     os.remove("./codex_log/{attribute}_log.txt".format(attribute=attribute))
    with tqdm(total=len(activate_list)) as pbar:
        while True:
            temp_pred_label ={}
            try:
                with open(output_log,"a") as file:
                    file.write("prompting begin at {count}\n".format(count=count))
                    
                test_story = dev_dataset[activate_list[count]]

                    ###
                for story_index in range(1,3):
                    
                    story_key = "story{story_index}".format(story_index=story_index)
                    entity_key = "entity_{story_index}".format(story_index=story_index)

                    entity_list = test_story[entity_key]
                    sentence_list = test_story[story_key]
                    InputPrompting = ''
                    InputPrompting = Prompting +getStoryPrompting_CSKG(sentence_list,entity_list)
                    Pred_label = np.zeros([len(entity_list),3])

                    response = openai.Completion.create(
                      model="code-davinci-002",
                      prompt= InputPrompting,
                      temperature=0,
                      max_tokens=1000,
                      top_p=1,
                      frequency_penalty=0,
                      presence_penalty=0,
                      stop = '#END'
                    )

                    response_clean = response['choices'][0]['text'].split(':')[1:]

                    for sentence_index,sub_response in enumerate(response_clean):
                        sub_response = sub_response.split("\n\t\t")[1:]
                        sub_label = []
                        if 'pass\n' in sub_response[0]:
                            continue
                        else:
                            for entity_index,state_response in enumerate(sub_response):
                                if entity_index == len(sub_response)-1:  ### the last item in slash n
                                    state_response = state_response.split("\n")[0]
                                entity_name = state_response.split('.')[0].strip()
                                if human_flag:
                                    if entity_name not in Name_list:
                                        continue
                                    else:
                                        entity_attribute = state_response.split('.')[1]
                                        if entity_attribute==attribute and entity_name in entity_list:
                                            sentence = sentence_list[sentence_index]
                                            attribute_label = singleAttributePrompting(sentence,entity_name,human_flag,output_log)
                                            Pred_label[entity_list.index(entity_name)][sentence_index] = attribute_label
                                else:
                                    if entity_name in Name_list:
                                        continue
                                    else:
                                        entity_attribute = state_response.split('.')[1]
                                        if entity_attribute==attribute and entity_name in entity_list:
                                            sentence = sentence_list[sentence_index]
                                            attribute_label = singleAttributePrompting(sentence,entity_name,human_flag,output_log)
                                            Pred_label[entity_list.index(entity_name)][sentence_index] = attribute_label
                                        

                    ###
                    temp_pred_label[story_index] = Pred_label
                count +=1
                pbar.update(1)
                for key,value in temp_pred_label.items():
                    Pred_summary.append(value)
                if count == len(activate_list):
                    break

            except Exception as e:
                with open(output_log.format(attribute=attribute),"a") as file:
                    file.write("reach limitation at {count}\n".format(count=count))
                    file.write("{e}\n".format(e=e))
                print("reach limitation at {count}".format(count=count))
                time.sleep(30)
                continue
                
    return Pred_summary

We use human attribute **conscious** as an example

In [38]:
summary_dict={}
folder_path = 'CSKG'
state='effects'
file_name= "{}_{}".format(folder_path,state)
log_name="{}_{}_log".format(folder_path,state)
mkdir(file_name)
mkdir(log_name)
human_flag = False
for attribute in ['conscious']:
    
# for attribute in ['clean']:
    attribute_label_list = list(att_adj[attribute])
    attribute_id = att_to_idx[attribute]
    if attribute_id < 5:
        human_flag = True
    activate_list = np.arange(len(cskg_data))[:2]
#     attribute_sen = get_attribute_dataset(tiered_dataset,Name_list,attribute_id,state)



    if attribute == 'location':
        Prompting = getProProming(attribute,'_',2,tiered_dataset,state)
    else:
        Prompting = getProProming(attribute,'_',6,tiered_dataset,state)
    
    
    Pred_summary=PromptingPipe_CSKG(cskg_data,attribute,activate_list,attribute_label_list,log_name,human_flag)

    folder_path = 'CSKG'
    output_file = os.path.join(file_name,attribute+".npy")
    np.save(output_file,Pred_summary)

Folder exist
Folder exist


 50%|█████     | 1/2 [00:26<00:26, 26.94s/it]

error! wait 30s


100%|██████████| 2/2 [01:31<00:00, 45.59s/it]


### ROCStories

loading data

In [40]:
roc_data = np.load('../Participant_Extraction/Participant/ROC_Participant.npy',allow_pickle = True)

In [46]:
for sample in roc_data:
    list1=sorted(sample['entity_1'])
    list2=sorted(sample['entity_2'])
    common_list=set(list1)&set(list2)
    common_list=sorted(list(common_list))
    sample['common_entity']=len(common_list)

main pipeline for prompting

In [48]:
def PromptingPipe_ROC(roc_data,attribute,activate_list,attribute_label_list,log_name,human_flag = False):
    attribute_id = att_to_idx[attribute]
    Pred_summary = []
    count = 0
    output_log = os.path.join(log_name,"{}_log.txt".format(attribute))
    with tqdm(total=len(activate_list)) as pbar:
        while True:
            temp_pred_label ={}
            with open(output_log,"a") as file:
                file.write("prompting begin at {count}\n".format(count=count))

            test_story = roc_data[activate_list[count]]
            if len(test_story['entity_1'])*len(test_story['entity_2'])==0:
                with open(output_log,"a") as file:
                    file.write("skip at {count}\n".format(count=count))                    
                count +=1
                pbar.update(1)
                Pred_label = np.zeros([len(entity_list),5])
                Pred_summary.append(Pred_label)
                Pred_summary.append(Pred_label)
                if count == len(activate_list):
                    break
                continue

            try:                    
                for story_index in range(1,3):
                    
                    story_key = "goal_sol_{story_index}".format(story_index=story_index)
                    entity_key = "entity_{story_index}".format(story_index=story_index)
                    common_entity = test_story['common_entity']
                    entity_list = list(test_story[entity_key])
                    sentence_list = test_story[story_key]
                    InputPrompting = ''
                    InputPrompting = Prompting +getStoryPrompting_CSKG(sentence_list,entity_list)
                    Pred_label = np.zeros([len(entity_list),5])
                    if story_index == 2 and common_entity >0:
                        Pred_label[:common_entity,:4] = temp_pred_label[1][:common_entity,:4]

                    response = openai.Completion.create(
                      model="code-davinci-002",
                      prompt= InputPrompting,
                      temperature=0,
                      max_tokens=1000,
                      top_p=1,
                      frequency_penalty=0,
                      presence_penalty=0,
                      stop = '#END'
                    )

                    response_clean = response['choices'][0]['text'].split(':\n\t\t')[1:]

                    for sentence_index,sub_response in enumerate(response_clean):
                        sub_response = sub_response.split("\n\t\t")
                        sub_label = []
                        if story_index==2 and sentence_index <4:
#                             print("SKIP at {}".format(sentence_index))
                            continue
                        if 'pass\n' in sub_response[0]:
                            continue
                        else:
                            for entity_index,state_response in enumerate(sub_response):
                                if entity_index == len(sub_response)-1:  ### the last item in slash n
                                    state_response = state_response.split("\n")[0]
                                entity_name = state_response.split('.')[0].strip()
                                if human_flag:
                                    if entity_name not in Name_list:
                                        continue
                                    else:
                                        entity_attribute = state_response.split('.')[1]
                                        if entity_attribute==attribute and entity_name in entity_list:
                                            sentence = sentence_list[sentence_index]
                                            attribute_label = singleAttributePrompting(sentence,entity_name,human_flag,output_log)
                                            Pred_label[entity_list.index(entity_name)][sentence_index] = attribute_label
                                else:
                                    if entity_name in Name_list:
                                        continue
                                    else:
                                        entity_attribute = state_response.split('.')[1]
                                        if entity_attribute==attribute and entity_name in entity_list:
                                            sentence = sentence_list[sentence_index]
                                            attribute_label = singleAttributePrompting(sentence,entity_name,human_flag,output_log)
                                            Pred_label[entity_list.index(entity_name)][sentence_index] = attribute_label
                                        

                    ###
                    temp_pred_label[story_index] = Pred_label
                count +=1
                pbar.update(1)
                for key,value in temp_pred_label.items():
                    Pred_summary.append(value)
                if count == len(activate_list):
                    break

            except Exception as e:
                with open(output_log,"a") as file:
                    file.write("reach limitation at {count}\n".format(count=count))
                    file.write("{e}\n".format(e=e))
                print("reach limitation at {count}".format(count=count))
                time.sleep(30)
                continue
                
    return Pred_summary

We use human attribute **conscious** as an example

In [49]:
summary_dict={}
folder_path = 'ROC'
state='effects'
file_name= "{}_{}".format(folder_path,state)
log_name="{}_{}_log".format(folder_path,state)
mkdir(file_name)
mkdir(log_name)
human_flag = False
for attribute in ['conscious']:
    
# for attribute in ['clean']:
    attribute_label_list = list(att_adj[attribute])
    attribute_id = att_to_idx[attribute]
    if attribute_id < 5:
        human_flag = True
    activate_list = np.arange(len(cskg_data))[:2]
#     attribute_sen = get_attribute_dataset(tiered_dataset,Name_list,attribute_id,state)



    if attribute == 'location':
        Prompting = getProProming(attribute,'_',2,tiered_dataset,state)
    else:
        Prompting = getProProming(attribute,'_',6,tiered_dataset,state)
    
    
    Pred_summary=PromptingPipe_ROC(roc_data,attribute,activate_list,attribute_label_list,log_name,human_flag)

    output_file = os.path.join(file_name,attribute+".npy")
    np.save(output_file,Pred_summary)

Folder exist
Folder exist


  0%|          | 0/2 [00:00<?, ?it/s]

error! wait 30s


100%|██████████| 2/2 [02:09<00:00, 64.85s/it]


In [50]:
Pred_summary

[array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0.]])]